In [15]:
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18(2)

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [28]:
ex1 = torch.rand(1,512)
ex2 = torch.rand(10, 3,1,512)

for j in range(10):
    for i in range(3):
        ex2[j][i] = ex1 * j

print(ex2.shape)
print(model.conv1(ex2))
print(out.size())

torch.Size([10, 3, 1, 512])
tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         ...,

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]],


        [[[-1.1579e+00, -1.5966e+00, -1.0039e+00,  ..., -1.6300e+00,
           -1.1021e+00, -9.4381e-01]],

         [[-5.8569e-01, -8.2779e-01, -7.5185e-01,  ..., -1.0102e+00,
           -8.4135e-01, -5.6000e-01]],

         [[ 8.6518e-08,  2.6065e-07,  5.0395e-07,  ...,  3.9816e-07,
            3.

In [19]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

In [5]:
print(sm.eval())

RecursiveScriptModule(original_name=MyModule)


In [ ]:
traced_script_module.save("traced_resnet_model.pt")

In [ ]:
%%writefile example-app.cpp

#include <torch/script.h> // One-stop header.

#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
  if (argc != 2) {
    std::cerr << "usage: example-app <path-to-exported-script-module>\n";
    return -1;
  }


  torch::jit::script::Module module;
  try {
    // Deserialize the ScriptModule from a file using torch::jit::load().
    module = torch::jit::load(argv[1]);
  }
  catch (const c10::Error& e) {
    std::cerr << "error loading the model\n";
    return -1;
  }

  std::cout << "ok\n";
}

Writing example-app.cpp


In [ ]:
!ls

example-app.cpp  sample_data  traced_resnet_model.pt


In [ ]:
!wget https://download.pytorch.org/libtorch/cu102/libtorch-win-shared-with-deps-debug-1.8.1%2Bcu102.zip

--2021-05-06 12:30:55--  https://download.pytorch.org/libtorch/cu102/libtorch-win-shared-with-deps-debug-1.8.1%2Bcu102.zip
Resolving download.pytorch.org (download.pytorch.org)... 99.84.219.100, 99.84.219.15, 99.84.219.59, ...
Connecting to download.pytorch.org (download.pytorch.org)|99.84.219.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1976582103 (1.8G) [application/zip]
Saving to: ‘libtorch-win-shared-with-deps-debug-1.8.1+cu102.zip’

libtorch-win-shared 100%[===================>]   1.84G  45.7MB/s    in 44s     

2021-05-06 12:31:40 (42.5 MB/s) - ‘libtorch-win-shared-with-deps-debug-1.8.1+cu102.zip’ saved [1976582103/1976582103]



In [ ]:
!unzip libtorch-win-shared-with-deps-debug-1.8.1+cu102.zip

Archive:  libtorch-win-shared-with-deps-debug-1.8.1+cu102.zip
   creating: libtorch/bin/
  inflating: libtorch/bin/protoc.exe  
 extracting: libtorch/build-hash     
   creating: libtorch/cmake/
  inflating: libtorch/cmake/protobuf-config-version.cmake  
  inflating: libtorch/cmake/protobuf-config.cmake  
  inflating: libtorch/cmake/protobuf-module.cmake  
  inflating: libtorch/cmake/protobuf-options.cmake  
  inflating: libtorch/cmake/protobuf-targets-debug.cmake  
  inflating: libtorch/cmake/protobuf-targets.cmake  
   creating: libtorch/include/
   creating: libtorch/include/asmjit/
  inflating: libtorch/include/asmjit/asmjit.h  
   creating: libtorch/include/asmjit/core/
  inflating: libtorch/include/asmjit/core.h  
  inflating: libtorch/include/asmjit/core/api-config.h  
  inflating: libtorch/include/asmjit/core/arch.h  
  inflating: libtorch/include/asmjit/core/assembler.h  
  inflating: libtorch/include/asmjit/core/builder.h  
  inflating: libtorch/include/asmjit/core/callconv.h

In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(custom_ops)

find_package(Torch REQUIRED)

add_executable(example-app example-app.cpp)
target_link_libraries(example-app "${TORCH_LIBRARIES}")
set_property(TARGET example-app PROPERTY CXX_STANDARD 14)

Writing CMakeLists.txt


rmdir: invalid option -- 'f'
Try 'rmdir --help' for more information.
